# Classification of DNA Sequences to identify invasive species with semi-supervised training

## Machine Learning at Berkeley Research Project

### Background

We attempt to solve the classification problem of identifying invasive species given binary labels and a DNA dataset from the island of Morea.

### Method

We separate training and testing data completely. Otherwise, same as `SVC_Final.ipynb`

## Data processing 

In [151]:
# import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sklearn as sk
from __future__ import division
from sklearn.cross_validation import train_test_split
import math

from tqdm import trange
from sklearn.svm import SVC

Here, we use pandas to read the excel sheet, and then extract features and convert the data to numpy arrays.

In [152]:
# read the excel sheet 
df = pd.read_excel('./BioCode for Machine Learning.xlsx')

# Read in the labels
cls = df['Classification']

# Read the DNA sequences, which are strings comprised of the letters ATCG
seq = df['Aligned Sequence']

The data we're working with are snippets of DNA a few hundred bases long.

In [153]:
seq[0]

'ACTTTATATTTTCTATTTGGAACATGAGCTGGAATAGTAGGAACATCTCTAAGA---ATTTTAATTCGTGCAGAACTTGGACATCCA---GGAGCTTTA------ATTGGAGATGATCAAATTTATAATGTAATTGTAACAGCTCATGCTTTTGTAATAATTTTTTTTATAGTAATACCTATTATAATTGGAGGATTTGGAAATTGATTAGTTCCTTTAATACTTGGAGCTCCTGATATAGCTTTTCCTCGAATAAACAATATAAGTTTTTGATTATTACCACCTTCTTTAACTTTATTATTAGTAAGAAGAATAGTTGAAAATGGAGCAGGAACTGGTTGAACAGTTTACCCTCCTCTTTCTGCTAGAATTGCACATGGAGGTGCATCTGTTGATTTAGCTATTTTTTCTCTTCATTTAGCTGGTATATCATCAATTTTAGGAGCAGTAAATTTTATTACTACAGTAATTAATATACGATCAAATGGAATTTCATATGATCGTATACCTTTATTTGTATGATCAGTTGTAATTACAGCTTTATTATTATTATTATCTTTACCTGTATTAGCAGGAGCTATTACTATACTTTTAACAGATCGAAATCTAAATACCTCATTTTTTGATCCTGCTGGAGGAGGAGATCCTATTTTATATCAACATTTATTT--------------------------------'

We can visualize the labels below. As we can see, the labels are very messy. We can only use the values of `Indigenous`, `Invasive`, or `NaN` for supervised training. However, because most unlabeled data points still have an associated DNA sequence, we can still use them in an unsupervised pre-training stage.

In [154]:
print(cls[:20])

0     Indigenous
1     Indigenous
2              0
3            NaN
4              0
5     Indigenous
6            NaN
7            NaN
8     Indigenous
9     Indigenous
10             0
11             0
12             0
13             0
14      Invasive
15    Introduced
16    Introduced
17             0
18    Introduced
19    Introduced
Name: Classification, dtype: object


However, some species don't even have associated DNA sequences. We have to discard these before we proceed.

In [155]:
# Shuffles the data (to make sure)
#cls = cls.sample(frac=1).reset_index(drop=True)

In [156]:
# Convert DNA data to numpy array, and convert NaNs to Nones

seq = np.array(seq.fillna('None'))

# Create a binary filter to eliminate invalid DNA sequences
valid_idx = np.array([i for i in range(len(seq)) if seq[i] != 'None'])

# Apply the filter
valid_seq = seq[valid_idx]
cls_valid = cls[valid_idx]
cls_valid = np.array(cls_valid)

Now, we process the DNA sequences by converting the string of bases into an array of characters.

In [157]:
valid_seq[0]

'ACTTTATATTTTCTATTTGGAACATGAGCTGGAATAGTAGGAACATCTCTAAGA---ATTTTAATTCGTGCAGAACTTGGACATCCA---GGAGCTTTA------ATTGGAGATGATCAAATTTATAATGTAATTGTAACAGCTCATGCTTTTGTAATAATTTTTTTTATAGTAATACCTATTATAATTGGAGGATTTGGAAATTGATTAGTTCCTTTAATACTTGGAGCTCCTGATATAGCTTTTCCTCGAATAAACAATATAAGTTTTTGATTATTACCACCTTCTTTAACTTTATTATTAGTAAGAAGAATAGTTGAAAATGGAGCAGGAACTGGTTGAACAGTTTACCCTCCTCTTTCTGCTAGAATTGCACATGGAGGTGCATCTGTTGATTTAGCTATTTTTTCTCTTCATTTAGCTGGTATATCATCAATTTTAGGAGCAGTAAATTTTATTACTACAGTAATTAATATACGATCAAATGGAATTTCATATGATCGTATACCTTTATTTGTATGATCAGTTGTAATTACAGCTTTATTATTATTATTATCTTTACCTGTATTAGCAGGAGCTATTACTATACTTTTAACAGATCGAAATCTAAATACCTCATTTTTTGATCCTGCTGGAGGAGGAGATCCTATTTTATATCAACATTTATTT--------------------------------'

In [158]:
# Seperate string into individual bases. So, each value in the array is a base. Stored in list
seq_arr = [np.array([i for i in s]) for s in valid_seq]

#seq_mtx = len(seq_arr)

print(len(valid_seq), len(cls_valid)) #, seq_mtx)

4639 4639


In [159]:
valid_labels = ['Introduced', 'Invasive', 'Indigenous']
cls_labeled = [label in valid_labels for label in cls_valid]
#labeled_cls = (valid_labels[labeled_cls] == 'Indigenous').astype(int)

# Create a filter telling us which points are valid to use for supervised training
cls_labeled = np.array(cls_labeled)
#print(len(labeled_cls))
unshuffled_labels = cls_valid[cls_labeled]

print(type(seq_arr))

seq_data = [i for i, validity in zip(seq_arr, cls_labeled) if validity]

print(type(seq_data))
print(len(seq_data))

<class 'list'>
<class 'list'>
1684


In [160]:
cls_valid_shuff, seq_data_shuff = sk.utils.shuffle(unshuffled_labels, seq_data, random_state=1337)

print(len(cls_valid_shuff))

# cls_train, cls_test, res_train, res_test = train_test_split(cls_valid_shuff, res_mat_shuff, test_size=0.2)
train_test_split = int(len(cls_valid_shuff)*0.2)

print(train_test_split)

cls_test, cls_train = cls_valid_shuff[:train_test_split], cls_valid_shuff[train_test_split:]

print(len(cls_test), len(cls_train))

1684
336
336 1348


In [161]:
cls_unlabeled = 1 - cls_labeled

seq_data_unlabeled = [i for i, validity in zip(seq_arr, cls_unlabeled) if validity]

print(len(seq_data_unlabeled))

# print(np.mean(unlabled_cls))

2955


In [162]:
num_unlabeled = np.sum(cls_unlabeled)
print(num_unlabeled)

2955


## Unsupervised Training

We create a similarity matrix, which is a pairwise comparison of DNA sequences and determining the percentage of base pairs that are the same.

Because the DNA sequences have been pre-aligned, we can expect this to be mostly accurate and close to the true similiarity values. In some places, the DNA sequences have a '-' character where the base was not read correctly, or missed. We ignore these.

In [163]:
sim_train = np.vstack((seq_data_unlabeled, seq_data_shuff[train_test_split:]))
print(sim_train.shape)

mat_size = len(sim_train)
print(mat_size)

sim_mat_train = -np.ones((mat_size, mat_size))

(4303, 701)
4303


In [164]:
# Precompute no dashes
dashes_train = []
for i in range(mat_size):
    dashes_train.append(sim_train[i] != '-')

In [165]:
print(sim_mat_train.shape)

(4303, 4303)


In [178]:
try:
    np.load('sim_mat_train.npy')

except:
    # this will take a few minutes
    for i in trange(mat_size):
        # clean up bad data
        a = sim_train[i]
        # iterate over DNA sequences and figure out the match
        for j in range(i):
            b = sim_train[j]
            match = (a==b)
            valid = (dashes_train[i] * dashes_train[j])
            sim_mat_train[i,j] = np.mean(match[valid])
            sim_mat_train[j,i] = sim_mat_train[i,j]
        sim_mat_train[i,i] = 1
    np.save('sim_mat_train.npy', sim_mat_train)

In [169]:
valid_mat_train = sim_mat_train

In [170]:
sim_mat_train.shape, valid_mat_train.shape

((4303, 4303), (4303, 4303))

The similarity matrix is very big (100mb+), so we try PCA/SVD to extract the most useful features from the largest singular values.

In [171]:
# %%time
# u,s,v = np.linalg.svd(valid_mat, full_matrices=0)

The rank / number of singular values we pick is a hyperparameter. We run the dimension reduction step.

In [172]:
# rank = 1000
# approx_1000 = u[:,:rank].dot(np.diag(s[:rank])).dot(v[:rank])
# errors = ((approx_1000 - valid_mat)/valid_mat)
# plt.hist(errors.flatten())

## Test data projection

We need to get the test data, which is currently still DNA bases, into sim matrix rows.

In [173]:
#sim_test = np.vstack((seq_arr_unlabeled, seq_arr_shuff[:train_test_split]))
sim_test = np.array(seq_data_shuff[:train_test_split])

sim_mat_test = -np.ones((mat_size, len(sim_test)))

print(sim_test.shape)
print(sim_mat_test.shape)

(336, 701)
(4303, 336)


In [174]:
# Precompute no dashes
dashes_test = []
for j in range(len(sim_test)):
    dashes_test.append(sim_test[j] != '-')

In [175]:
print(len(dashes_test))

336


In [176]:
# this will take a few minutes
for i in trange(mat_size):
    # clean up bad data
    a = sim_train[i]
    # iterate over DNA sequences and figure out the match
    for j in range(len(sim_test)):
        b = sim_test[j]
        match = (a==b)
        valid = (dashes_train[i] * dashes_test[j])
        sim_mat_test[i,j] = np.mean(match[valid])
        # sim_mat_test[j,i] = sim_mat_test[i,j]
    # sim_mat_test[i,i] = 1

100%|██████████| 4303/4303 [00:46<00:00, 92.92it/s] 


In [179]:
print(sim_mat_test)

[[ 0.77947933  0.77758621  0.69072165 ...,  0.75990676  0.79173047
   0.79019908]
 [ 0.79299848  0.78253425  0.70360825 ...,  0.78554779  0.80060883
   0.81582953]
 [ 0.87975647  0.81164384  0.67783505 ...,  0.80652681  0.89497717
   0.8934551 ]
 ..., 
 [ 0.85502959  0.83136095  0.66541353 ...,  0.80909091  0.87869822
   0.85207101]
 [ 0.7412481   0.72431507  0.67525773 ...,  0.70862471  0.75799087
   0.75038052]
 [ 0.75342466  0.74315068  0.64948454 ...,  0.69230769  0.75038052
   0.74885845]]


In [180]:
sim_mat_test.shape

(4303, 336)

## Supervised Training

Now, we have a set of features from our pre-training step, and we're ready to run supervised training. Before we start, we need to first remove data that don't have valid labels. We can't use them anymore!

We see that 36% of the data have associated labels.

In [181]:
np.mean(cls_labeled)

0.36300926923906013

In [182]:
print(supervised_y)

[1 1 0 ..., 0 0 0]


In [189]:
# apply the filter over our features and labels
# supervised_X = approx[labeled_cls]
supervised_X = valid_mat_train[num_unlabeled:]
# supervised_y = cls_valid[labeled_cls]

supervised_y_train = (cls_train == 'Indigenous').astype(int)

supervised_y_test = (cls_test == 'Indigenous').astype(int)

print(supervised_y_train.shape, supervised_y_test.shape)

(1348,) (336,)


Below, we run Support Vector Clustering (SVC). We shuffle the data first, and then split our data into testing and training splits.

There is somewhat large variance inbetween runs, so we take the average for a more accurate score.

In [190]:
# 10/18

c = 1e15

avg_score = []


for _ in range(2):
#     res_mat_shuff, cls_valid_shuff = sk.utils.shuffle(supervised_X, supervised_y, random_state=0)

#     cls_train, cls_test, res_train, res_test = train_test_split(cls_valid_shuff, res_mat_shuff, test_size=0.2)

    # print(len(cls_train), len(cls_test))

    X = supervised_X
    y = supervised_y_train #.reshape(-1, 1)
    X_test = sim_mat_test.T
    
    print(X.shape, y.shape, X_test.shape)

    clf = SVC(C=c, kernel='poly', degree=2, coef0=0)

    clf.fit(X, y)

    predict = clf.predict(X_test)
    
    print(predict[:20])
    # print(predict == np.array(cls_test))
    print(cls_test[:20])
    score = np.mean((predict == np.array(supervised_y_test))*1)
    avg_score.append(score)

print(avg_score, np.mean(avg_score))

# print 'Approximated similarity matrix: \n'
# test_and_score(supervised_X, supervised_y)
# print 'Full similarity matrix: \n'
# test_and_score(full_supervised_X, supervised_y)

(1348, 4303) (1348,) (336, 4303)
[0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0]
['Invasive' 'Introduced' 'Introduced' 'Introduced' 'Invasive' 'Indigenous'
 'Indigenous' 'Indigenous' 'Invasive' 'Introduced' 'Invasive' 'Invasive'
 'Indigenous' 'Introduced' 'Indigenous' 'Invasive' 'Invasive' 'Indigenous'
 'Introduced' 'Invasive']
(1348, 4303) (1348,) (336, 4303)
[0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0]
['Invasive' 'Introduced' 'Introduced' 'Introduced' 'Invasive' 'Indigenous'
 'Indigenous' 'Indigenous' 'Invasive' 'Introduced' 'Invasive' 'Invasive'
 'Indigenous' 'Introduced' 'Indigenous' 'Invasive' 'Invasive' 'Indigenous'
 'Introduced' 'Invasive']
[0.9821428571428571, 0.9821428571428571] 0.982142857143


We see the results are very competitive.

Now, we use cross validation method get another take on our performance.

In [101]:
# # cross validation method
# # SVM

# # tricks: shuffling data, cross validation, balanced classes, hyperparam tuning

# def cv_test_and_score(supervised_X_train, supervised_y, c=1e14):
#     scores = []
#     param_vals = []
    
#     # shuffle the data
#     # res_mat_shuff, cls_valid_shuff = sk.utils.shuffle(supervised_X, supervised_y, random_state=0)

#     c = 10*c
#     clf = SVC(C=c,kernel='poly', degree=2, coef0=0) #, gamma=i)

#     score = sk.cross_validation.cross_val_score(clf, res_mat_shuff, cls_valid_shuff, cv=6) #, n_jobs=-1)
#     # print('Prediction accuracy:', np.mean((prediction == np.array(cls_test))*1))
#     #Coefficients used by the classifier

#     scores.append(score)
#     param_vals.append(i)

#     print(scores)

#     mn_scores = [np.mean(score) for score in scores]

#     print('mean scores:', mn_scores)

# print ('Approximated similarity matrix: \n')
# cv_test_and_score(supervised_X, supervised_y)
# # print ('\nFull similarity matrix: \n')
# # cv_test_and_score(full_supervised_X, supervised_y)

Notice that we included our testing data when creating the similiarity matrix, because we first create the matrix and then separate the data into train and test sets. This is somewhat unsatisfying, and very anonying if we want to do on the fly predictions. We have to recompute the simliarity matrix every time.

We now try excluding the test data from computing the similarity matrix. Instead, we can compute the values for the test data afterwards. We then also need to project the similiarity values for the test data to the SVD space, before we can run SVC.